In [ ]:
from delta.tables import *
from notebookutils import mssparkutils
from datetime import datetime, timedelta
from pyspark.sql.functions import col, when, from_json, date_format, lit, row_number,max, lower, to_date, regexp_replace
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType, StringType
from dateutil.relativedelta import relativedelta
import requests
import os
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true") # needed for automatic schema evolution in merge

StatementMeta(, d61c0051-47a3-459e-ac4d-371c22968bd8, 22, Finished, Available, Finished)

In [21]:
fromMonth = -2 #-1, -2,... from datenow -1 day 
toMonth = -1 #-1, -2,... from datenow -1 day

StatementMeta(, d61c0051-47a3-459e-ac4d-371c22968bd8, 23, Finished, Available, Finished)

In [22]:
rawSourcePath = "Files/reservation-transactions"

StatementMeta(, d61c0051-47a3-459e-ac4d-371c22968bd8, 24, Finished, Available, Finished)

In [23]:
def find_first_csv_file(path):
    """
    Recursively search for the first .csv file in the given directory.
    Args:
        path (str): The root directory to start the search.
    Returns:
        str or None: The full path to the first .parquet file found, or None if not found.
    """
    try:
        for entry in mssparkutils.fs.ls(path):
            if entry.isFile and entry.name.endswith(".csv"):
                return entry.path
            elif entry.isDir:
                result = find_first_csv_file(entry.path)
                if result:
                    return result
    except Exception as e:
        print(f"Error accessing {path}: {e}")
    return None

def generate_wildcard_path(full_path: str, raw_source_path: str, snapshot_folder: str) -> str:
    # Find the index where the raw source path starts
    idx = full_path.find(raw_source_path)
    if idx == -1:
        raise ValueError("rawSourcePath not found in full path")

    # Extract the base URI before the raw source path
    base_uri = full_path[:idx]
    detailPath = structurePath[idx+len(rawSourcePath):]

    startleveltoAdd = detailPath.count('/') - '/date-date/Guid/name.csv'.count('/')

    # Construct the wildcard path
    wildcard_path = f"{base_uri}{raw_source_path}{'/*' * startleveltoAdd}/{snapshot_folder}/*.csv"
    return wildcard_path

def generateArrayOFPeriod(from_Month: int, to_month: int):
    # Get today's date
    today = datetime.today()

    # Subtract one day
    yesterday = today - timedelta(days=1)
    first_day = yesterday.replace(day=1)

    periodToLoad = []
    if from_Month == to_month:
        periodDate = first_day + relativedelta(months=to_month)
        periodToLoad.append(periodDate.date())
    else:
        for i in range(from_Month, to_month+1):
            periodDate = first_day + relativedelta(months=i)
            periodToLoad.append(periodDate.date())

    return periodToLoad

StatementMeta(, d61c0051-47a3-459e-ac4d-371c22968bd8, 25, Finished, Available, Finished)

# Step1: Load Silver

In [24]:
structurePath = find_first_csv_file(rawSourcePath)
periodsToLoad = generateArrayOFPeriod(fromMonth, toMonth)

StatementMeta(, d61c0051-47a3-459e-ac4d-371c22968bd8, 26, Finished, Available, Finished)

In [25]:
for per in periodsToLoad:
    print("Start Period : " + per.strftime("%Y-%m-%d"))

    #drop staging table if exists
    spark.sql("DROP TABLE IF EXISTS dim_reservations_staging")

    #generate storage path date part
    fromFormatedDate = per.strftime("%Y%m%d")
    toFormatedDate = (per + relativedelta(months=1) + relativedelta(days=-1)).strftime("%Y%m%d")
    snapshot_folder = fromFormatedDate + "-" + toFormatedDate

    wildcard = generate_wildcard_path(structurePath, rawSourcePath, snapshot_folder)
    print("Used path to load data: " + wildcard)

    try:
        df = spark.read.csv(wildcard, header=True, inferSchema=True)
        df= df.withColumn("PeriodLoaded",to_date(date_format("EventType", "yyyy-MM-01")) )
        df.write.format('delta').saveAsTable("dim_reservations_staging")

        #identify period loaded
        df = spark.sql("SELECT PeriodLoaded FROM dim_reservations_staging LIMIT 1")
        value = df.first()['PeriodLoaded']

        #clean existing data in silver
        if spark._jsparkSession.catalog().tableExists('dim_reservations_silver'):
            print("Table exists, snapshot will be clean.")
            spark.sql(f"DELETE FROM dim_reservations_silver WHERE PeriodLoaded = '{value}'")

        #Load data in silver
        dim_reservations_staging_df = DeltaTable.forPath(spark,"Tables/dim_reservations_staging").toDF()
        dim_reservations_staging_df.write.mode("append").option("mergeSchema", "true").format("delta").saveAsTable("dim_reservations_silver")
        print("Data loaded in silver.")

    except AnalysisException as e:
        if "PATH_NOT_FOUND" in str(e):
            print(f"Path not found: {wildcard}")
        else:
            raise # re-raise if it's a different AnalysisException

    print("End Period : " + per.strftime("%Y-%m-%d"))

StatementMeta(, d61c0051-47a3-459e-ac4d-371c22968bd8, 27, Finished, Available, Finished)

Start Period : 2025-08-01
Used path to load data: abfss://b5e66845-fd73-4d5d-b28c-da7dbc2d17ec@onelake.dfs.fabric.microsoft.com/e3e0ae4e-208c-4a38-bf39-9671d2e2e0e6/Files/reservation-transactions/20250801-20250831/*.csv
Table exists, snapshot will be clean.
Data loaded in silver.
End Period : 2025-08-01
Start Period : 2025-09-01
Used path to load data: abfss://b5e66845-fd73-4d5d-b28c-da7dbc2d17ec@onelake.dfs.fabric.microsoft.com/e3e0ae4e-208c-4a38-bf39-9671d2e2e0e6/Files/reservation-transactions/20250901-20250930/*.csv
Table exists, snapshot will be clean.
Data loaded in silver.
End Period : 2025-09-01


# Step2: Load to Gold Dim Reservation

In [26]:
source_df = DeltaTable.forPath(spark,"Tables/dim_reservations_silver").toDF()
source_df = source_df.where("lower(ArmSkuName) = 'fabric_capacity_cu_hour'")

StatementMeta(, d61c0051-47a3-459e-ac4d-371c22968bd8, 28, Finished, Available, Finished)

In [27]:
tableName = "reservations"
logicalKeyColumn = "ReservationOrderId"
technicalKeyColumn = "ReservationOrderKey"
tableAlreadyExists = spark._jsparkSession.catalog().tableExists('FCA', tableName)


source_merge_df = source_df\
                 .select("Amount","BillingFrequency","Currency","Description","EventDate","PurchasingSubscriptionGuid","PurchasingSubscriptionName","Quantity","Region","ReservationOrderId","ReservationOrderName")\
                 .distinct()

if tableAlreadyExists:
    #Merge to table

    print(f"Merge Data for {tableName} table Started")

    target_table = DeltaTable.forPath(spark, f"Tables/{tableName}")
    target_df = target_table.toDF()
    target_df = target_df.select(logicalKeyColumn,technicalKeyColumn)


    max_key = target_df.agg(max(technicalKeyColumn)).collect()[0][0]

    combined_df = source_merge_df.join(target_df,logicalKeyColumn,"leftouter")
    existingRows_df = combined_df.where(combined_df[technicalKeyColumn].isNotNull())
    newRows_df = combined_df.where(combined_df[technicalKeyColumn].isNull())
    window_spec = Window.orderBy(logicalKeyColumn)
    newRows_df = newRows_df.withColumn(technicalKeyColumn, row_number().over(window_spec) + max_key )

    Src_Merge_df = existingRows_df.union(newRows_df)


    merge = (target_table.alias("target")
        .merge(
            Src_Merge_df.alias("source"),
            f"target.{technicalKeyColumn} = source.{technicalKeyColumn}"
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        )
    merge.execute()

    print(f"Merge Data for {tableName} Ended")


else:
    print(f"Table {tableName} creation started")
    window_spec = Window.orderBy(logicalKeyColumn)
    source_merge_df = source_merge_df.withColumn(technicalKeyColumn, row_number().over(window_spec))
    source_merge_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable(tableName)
    print(f"Table {tableName} creation Ended")

StatementMeta(, d61c0051-47a3-459e-ac4d-371c22968bd8, 29, Finished, Available, Finished)

Merge Data for reservations table Started
Merge Data for reservations Ended
